### Setup

In [14]:
import numpy as np

from hot_dust import preprocess, model

In [15]:
from importlib import reload

preprocess = reload(preprocess)
model = reload(model)

In [16]:
ds = preprocess.prepare_training_data()
train, validate, test = preprocess.to_tensorflow(preprocess.split_training_data(ds))

### Parameters

In [17]:
epochs = 500 # Max 500
batch_size = 64
buffer_size = 10 * batch_size

model = reload(model)  # DEBUG

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size)

layer = model.pretraining(dataset=test)

In [18]:
network = model.compile(normalization=layer)
network.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 11)]              0         
                                                                 
 normalization_2 (Normalizat  (None, 11)               23        
 ion)                                                            
                                                                 
 dense_6 (Dense)             (None, 8)                 96        
                                                                 
 dense_7 (Dense)             (None, 8)                 72        
                                                                 
 dense_8 (Dense)             (None, 1)                 9         
                                                                 
Total params: 200
Trainable params: 177
Non-trainable params: 23
____________________________________________________________

### Fitting

In [19]:
import tensorflow as tf 
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", fit)

Epoch 1/500


3122/3122 - 4s - loss: 0.7281 - val_loss: 0.4430 - 4s/epoch - 1ms/step
Epoch 2/500
3122/3122 - 3s - loss: 0.3877 - val_loss: 0.3209 - 3s/epoch - 1ms/step
Epoch 3/500
3122/3122 - 3s - loss: 0.2892 - val_loss: 0.2634 - 3s/epoch - 833us/step
Epoch 4/500
3122/3122 - 3s - loss: 0.2488 - val_loss: 0.2346 - 3s/epoch - 889us/step
Epoch 5/500
3122/3122 - 3s - loss: 0.2274 - val_loss: 0.2176 - 3s/epoch - 833us/step
Epoch 6/500
3122/3122 - 3s - loss: 0.2126 - val_loss: 0.2045 - 3s/epoch - 812us/step
Epoch 7/500
3122/3122 - 3s - loss: 0.2004 - val_loss: 0.1926 - 3s/epoch - 838us/step
Epoch 8/500
3122/3122 - 3s - loss: 0.1893 - val_loss: 0.1835 - 3s/epoch - 838us/step
Epoch 9/500
3122/3122 - 3s - loss: 0.1804 - val_loss: 0.1755 - 3s/epoch - 914us/step
Epoch 10/500
3122/3122 - 3s - loss: 0.1738 - val_loss: 0.1701 - 3s/epoch - 840us/step
Epoch 11/500
3122/3122 - 2s - loss: 0.1688 - val_loss: 0.1660 - 2s/epoch - 788us/step
Epoch 12/500
3122/3122 - 4s - loss: 0.1645 - val_loss: 0.1617 - 4s/epoch - 1ms/

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [20]:
model.plot_loss(fit)

:NdOverlay   [Variable]
   :Scatter   [epoch]   (value)